In [ ]:
!pip install lyricsgenius
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from lyricsgenius import Genius

In [ ]:
url_dave_30s = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-of-1930-1939.html'

headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

response = requests.get(url_dave_30, headers=headers)

response.status_code

In [ ]:
soup = BeautifulSoup(response.content)

In [ ]:
tags

In [ ]:
#Obtengo un set de los tags presentes en el tag 'p'
tags = soup.find_all(name="p")

#Separa en elementos de una lista las canciones.
songs_30s = ''.join(i.text for i in tags[5:15]).split('\n')
songs_30 = tags[5].text.split('\n')
#Elimina los espacios de los extremos de cada elemento de la lista si
#los hubiese y devuelve una lista nueva con cada elemento
list_songs_30s = [i.strip() for i in songs_30s]

#Elimina los elementos vacios de la lista
list_songs_30s = list(filter(None, list_songs_30s))
#songs_30s = songs.split('\n')

list_songs_30s

In [ ]:
def list_songs_url(url):
    headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content)
    tags = soup.find_all(name="p")
    
    #Separa en elementos de una lista las canciones.
    songs = tags[5].text.split('\n')
    
    #Elimina los espacios de los extremos de cada elemento de la lista si
    #los hubiese y devuelve una lista nueva con cada elemento
    songs = [i.strip() for i in songs]
    
    #Elimina los elementos vacios de la lista
    songs = list(filter(None, songs))
    return songs

list_songs_url(url_dave)

In [ ]:
test_str = '81. The Peanut Vendor (El Manisero)...Don Azpiazu with Antonio Machin (1930)'

#Recibe un string y devuelve el a;o de el string de la cancion
def extract_year(song_string):
    return song_string[-5:-1]

extract_year(test_str)

In [ ]:
#Recibe un string y devuelve el a;o de el string de la cancion
def extract_decade(song_string):
    return extract_year(song_string)[:3] + '0'

extract_decade(test_str)

In [ ]:
#Recibe un string y devuelve el orden de la cancion
def extract_order(song_string):
    return song_string[:3].replace('.','').strip()

extract_order(test_str)

In [ ]:
#Recibe un string y devuelve el titulo de la cancion
def extract_title(song_string):
    return re.findall('\d+.\s(.*)(?:\.\.\.|\…)',song_string)[0]

extract_title(test_str)

In [ ]:
#Recibe un string y devuelve el autor de la cancion
def extract_author(song_string):
    return re.findall('(?:\.\.\.|\…)(.*)\(',song_string)[0].split('with')[0].strip()

extract_author(test_str)

In [ ]:
def ext_data_songs(url_dave):
    list_songs_dave = list_songs_url(url_dave)
    formated_songs = []
    for i in list_songs_dave:
        decade = extract_decade(i)
        year = extract_year(i)
        title = extract_title(i)
        author = extract_author(i)
        order = extract_order(i)
        
        song = {
            'decade':decade,
            'year':year,
            'title':title,
            'author':author,
            'order':order,
        }
        formated_songs.append(song)
    return formated_songs

ext_data_songs(url_dave_30)

In [ ]:
dicc_30s = ext_data_songs(url_dave_30)
df_30s = pd.DataFrame(dicc_30s)
df_30s

# Spotify

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="00e41fa4725d4469b29d9e36f43d7ec9",
                                                           client_secret="e64b15d24085495283f18937da5452d3"))

In [ ]:
def search_spoti(title, author):
    search = sp.search(title + ' ' + author, market='US')
    tracks = search['tracks']
    df = pd.json_normalize(tracks['items'])
    df = df[['id', 'name', 'popularity', 'artists', 'album.name', 'external_urls.spotify', 'uri', 'explicit', 'album.release_date', 'duration_ms']]
    df['artists'] = df.artists.apply(lambda d: d[0]['name'])
    return df.head(1)

In [ ]:
def songs_to_spoti_df(songs_df):
    lista = []

    for i in range(len(songs_df)):
        try:
            search = search_spoti(songs_df.loc[i, "title"], songs_df.loc[i, "author"])
            lista.append(search)
        except KeyError:
            pass
    
    return pd.concat(lista, axis=0)

songs_to_spoti_df(df_30s)

# Genius

In [ ]:
!pip install lyricsgenius

In [ ]:
import lyricsgenius
genius = lyricsgenius.Genius(
    'v3s3cdBYSN4Kta3F2HDTmdqqNOckKEZTQA5O5Bhb5J-BV_L02jqPGCEnvpKsSrBz')

In [ ]:
artist = genius.search_artist("Judy Garland", max_songs=3, sort="title")

In [ ]:
song = artist.song("Over the Rainbow")

In [ ]:
song.lyrics

In [ ]:
def get_lyrics(artist_name, song_name):
    genius = lyricsgenius.Genius(
    'v3s3cdBYSN4Kta3F2HDTmdqqNOckKEZTQA5O5Bhb5J-BV_L02jqPGCEnvpKsSrBz')
    artist = genius.search_artist(artist_name, max_songs=0, sort="title")
    song = artist.song(song_name)
    return re.sub("[\(\[].*?[\)\]]", "", song.lyrics.replace('\n', ' '))

In [ ]:
get_lyrics("acdc", "back in black")

In [ ]:
def get_df_with_lyrics(spoti_songs_df):
    lyrics_list = []

    for artist, song in zip(spoti_songs_df['artists'], spoti_songs_df['name']):
        try:
            lyrics_list.append(get_lyrics(artist, song))
        except AttributeError:
            lyrics_list.append('None')
    
    spoti_songs_df['lyrics'] = lyrics_list
    return spoti_songs_df

In [ ]:
get_df_with_lyrics(spoti_songs_df_30s)